Connect to cost control data and create revenue dataset

In [1]:
realm = 'temper'

In [2]:
import os

import mysql.connector
import pandas as pd

from datetime import datetime
import calendar

mydb = mysql.connector.connect(
  host=os.environ['UAT_RDS_CLONE_HOST'],
  user=os.environ['UAT_RDS_CLONE_USER'],
  password=os.environ['UAT_RDS_CLONE_PASSWORD'],
  database="rr_core"
)

In [3]:
# get data

myQuery = f'''-- Date,Open,High,Low,Close,Adj Close,Volume,UpperB,LowerB,PercentB

SELECT
	date_of_business,
	data_value as 'Open',
	max(data_value) as High,
	min(data_value) as Low,
	sum(data_value) as Close,
	sum(data_value) as AdjClose,
	count(id) as Volume,
	max(data_value) * 0.95 as UpperB,
	 max(data_value) * 0.05 as UpperB,
	max(data_value) / sum(data_value) as PercentB
FROM `collated_cc_costcontrol_data`
WHERE type_id = 2 and realm = '{realm}'
GROUP BY date_of_business
'''

df_coco = pd.read_sql(myQuery, con=mydb)

mydb.close()

df_coco.head() # TAKES 3 MINS

/var/folders/t6/ysrkn8dn65b6bsvj1sb28p9m0000gp/T/ipykernel_4066/1444831783.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_coco = pd.read_sql(myQuery, con=mydb)


,date_of_business,Open,High,Low,Close,AdjClose,Volume,UpperB,UpperB,PercentB
0,2022-02-28,2885.99,2885.99,254.86,4448.80,4448.80,4,2741.6905,144.2995,0.648712
1,2022-03-01,1549.74,3743.79,1549.74,10725.93,10725.93,4,3556.6005,187.1895,0.349041
2,2022-03-02,2759.32,7517.14,2759.32,21043.59,21043.59,4,7141.2830,375.8570,0.357218
3,2022-03-03,1750.21,5047.83,1750.21,13326.09,13326.09,4,4795.4385,252.3915,0.378793
4,2022-03-04,2116.42,4198.25,2116.42,6314.67,6314.67,2,3988.3375,209.9125,0.664841


In [4]:
df_coco.shape

(860, 10)

In [5]:
df_coco.columns

Index(['date_of_business', 'Open', 'High', 'Low', 'Close', 'AdjClose',
       'Volume', 'UpperB', 'UpperB', 'PercentB'],
      dtype='object')

In [7]:
df_coco.to_csv(f'./data/coco_{realm}.csv', index=False)